In [3]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [4]:
LOSS_FUNCTION = "categorical_crossentropy"
NUM_EPOCH = 10
BATCH = 64
OPTIMIZER ="Adam"


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,

    rotation_range=40,

    width_shift_range=0.2,

    height_shift_range=0.2,

    shear_range=0.2,

    zoom_range=0.2,

    horizontal_flip=True,

    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,

    rotation_range=40,

    width_shift_range=0.2,

    height_shift_range=0.2,

    shear_range=0.2,

    zoom_range=0.2,

    horizontal_flip=True,

    fill_mode='nearest'
)


TRAIN_DIRECTORY_LOCATION = 'leafs/train'
VAL_DIRECTORY_LOCATION = 'leafs/validation'
TARGET_SIZE = (500,500)
CLASS_MODE = 'categorical'

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIRECTORY_LOCATION,
    target_size = TARGET_SIZE,
    batch_size = 32,
    class_mode = CLASS_MODE
)

validation_generator = validation_datagen.flow_from_directory(
    VAL_DIRECTORY_LOCATION,
    target_size = TARGET_SIZE,
    batch_size = 32,
    class_mode = CLASS_MODE
)


Found 686 images belonging to 2 classes.
Found 88 images belonging to 2 classes.


In [6]:

model = tf.keras.models.Sequential([

   tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(500, 500, 3)),

   tf.keras.layers.MaxPooling2D(2, 2),

   tf.keras.layers.Conv2D(64, (3,3), activation='relu'),

   tf.keras.layers.MaxPooling2D(2,2), 
    
   tf.keras.layers.Conv2D(128, (3,3), activation='relu'),

   tf.keras.layers.MaxPooling2D(2,2),

   tf.keras.layers.Conv2D(64, (3,3), activation='relu'),

   tf.keras.layers.MaxPooling2D(2,2),

   tf.keras.layers.Flatten(),

   tf.keras.layers.Dense(512, activation='relu'),

   tf.keras.layers.Dense(2, activation='softmax')
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 498, 498, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 249, 249, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 247, 247, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 123, 123, 64)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 121, 121, 128)     73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 60, 60, 128)      0

In [7]:
model.compile(
    loss = LOSS_FUNCTION,
    optimizer = OPTIMIZER,
    metrics = ['accuracy']
)

model.fit(
      train_generator,
      epochs = NUM_EPOCH,
      verbose = 1,
      validation_data = validation_generator)

model.save(f"./yaprak.h5")


converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


Epoch 1/10
22/22 [==============================] - 227s 10s/step - loss: 1.0053 - accuracy: 0.5452 - val_loss: 0.6391 - val_accuracy: 0.7841
Epoch 2/10
22/22 [==============================] - 234s 11s/step - loss: 0.5567 - accuracy: 0.7478 - val_loss: 0.4682 - val_accuracy: 0.7841
Epoch 3/10
22/22 [==============================] - 242s 11s/step - loss: 0.5024 - accuracy: 0.7857 - val_loss: 0.4629 - val_accuracy: 0.7386
Epoch 4/10
22/22 [==============================] - 246s 11s/step - loss: 0.4305 - accuracy: 0.8090 - val_loss: 0.3819 - val_accuracy: 0.8295
Epoch 5/10
22/22 [==============================] - 249s 11s/step - loss: 0.3802 - accuracy: 0.8382 - val_loss: 0.2961 - val_accuracy: 0.8636
Epoch 6/10
22/22 [==============================] - 247s 11s/step - loss: 0.3989 - accuracy: 0.8163 - val_loss: 0.4566 - val_accuracy: 0.7500
Epoch 7/10
22/22 [==============================] - 248s 11s/step - loss: 0.3257 - accuracy: 0.8557 - val_loss: 0.2368 - val_accuracy: 0.9091
Epoch 

INFO:tensorflow:Assets written to: C:\Users\KEMALE~1\AppData\Local\Temp\tmpoil1fpfl\assets


INFO:tensorflow:Assets written to: C:\Users\KEMALE~1\AppData\Local\Temp\tmpoil1fpfl\assets


In [8]:
with open("model.tflite", "wb") as f:
    f.write(tflite_model)